In [2]:
import matplotlib.pyplot as plt
import os
import numpy as np
from imageio import imread, imsave
from skimage.color import rgb2gray
from skimage.feature import corner_harris, corner_peaks, corner_fast, corner_subpix, ORB
import cv2
import opencv as oc


В лабораторной необходимо по положению ручек стула определить, можно ли задвинуть стул под/над стол или такой возможности нет. 
Ручки стула обернуты в зеленую ткань для контраста с материалом стола и стула.

Изображения разложены по папкам с названиями, соответствующими результату.

In [3]:
# Directories here - change name(!)
dirs = ['C:\\Users\\Daria\\Documents\\Обработка сигналов\\Big_lab\\Up', 
        'C:\\Users\\Daria\\Documents\\Обработка сигналов\\Big_lab\\Low',
        'C:\\Users\\Daria\\Documents\\Обработка сигналов\\Big_lab\\Error']
name_dirs = ['Above', 'Below', 'Impossible']

Для определения границ столешницы используется детектор Хафа от границ изображения: необходимо выделить верхнюю и нижнюю грани столешницы (горизонтальные линии).
После применения детектора имеем расстояния от левого верхнего угла изображения и углы между горизонтальной осью и этим расстоянием, по ним вычисляем высоту (от верхнего левого угла до точки, соответвующей началу линии по оси ОУ).

In [4]:
from skimage.transform import (hough_line, hough_line_peaks,
                               probabilistic_hough_line)
from skimage.feature import canny
from matplotlib import cm


def show_hough_transform(image):
    h, theta, d = hough_line(canny(image))  # вычисляем преобразование Хафа от границ изображения
    
    # find horizontal lines (dist + angle)
    h_up = image.shape[0]
    dist_hor = []
    for _, angle, dist in zip(*hough_line_peaks(h, theta, d)):
        if np.abs(angle) > 1:
            dist_hor.append((np.abs(dist), angle))
    # need 2 smallest as nearest to peek horizontal
    sorted_dist = sorted(dist_hor, key=lambda dist: dist[0])
    # here is cos(pi/2 - angle)
    h_up = np.abs(sorted_dist[0][0] / np.sin(sorted_dist[0][1]))
    h_low = np.abs(sorted_dist[1][0] / np.sin(sorted_dist[1][1]))

    return h_up, h_low

Определяем особые точки с помощью SIFT для определения точек на ручках (для этого и используется контрастный цвет).

In [5]:
# SIFT for special points searching
def sift_с(img):
    sift = cv2.SIFT_create()
    # Keypoints and descriptors.
    kp1, des1 = sift.detectAndCompute(img, None)
    
    return kp1

Среди этих особых точек ищем зеленые (для этого заданы верхняя и нижняя границы зеленого цвета).
После прохода по особым точкам и отделения зеленых находим наименьшую и наибольшую высоту от края изображения.


In [26]:
def get_green_point_height(kp, img):
    # Convert key points to coordinates
    pts = cv2.KeyPoint_convert(kp)
    # Round coordinates to get pixel
    pts = np.round(pts, 0)

    #print(pts)    
    #print(pts[-1])
    
    # BGR is here
    green_min = np.array((10, 81, 7))
    green_max = np.array((115, 222,111))


    def is_green(b, g, r):
        if b >= green_min[0] and g >= green_min[1] and r >= green_min[2] and b <= green_max[0] and g <= green_max[1] and r <= green_max[2]:
            return True

    #print(img.shape)
    green_points_height = []
    for point in pts:
        if (int(point[0]) < img.shape[0] and int(point[1]) < img.shape[1]):
            (b, g, r) = img[int(point[0]), int(point[1])]
            if is_green(b, g, r):
                #print(int(point[0]), int(point[1]))
                green_points_height.append(int(point[0]))

    if len(green_points_height) == 0:
        return -1, -1
    max_h = np.min(green_points_height)
    min_h = np.max(green_points_height)

    #print(max_h, min_h)
    return max_h, min_h


Интересуют случай, когда верхняя точка зеленой ручки по высоте больше
нижнего края столешницы («пройдет ниже») и случай, когда нижняя точка меньше верхней 
грани столешницы («пройдет выше»), иначе получили «не пройдет».
Стоит отметить, что высота считается от верхнего края!

In [28]:
i = 0
for dir_ in dirs:
    images = [img_ for img_ in os.listdir(dir_)
              if img_.endswith(".jpg") or
              img_.endswith(".jpeg") or
              img_.endswith("png")]
    print(name_dirs[i])
    for j in range(len(images)):
        os.chdir(dir_)
        image = imread(images[j])
        gray = rgb2gray(image)
        table_max, table_min = show_hough_transform(gray)
        key_points = sift_с(image)
        chair_max, chair_min = get_green_point_height(key_points, image)
        print('Image ' + str(j) + ': ', end ="")
        if chair_max < 0 or chair_min < 0: 
            print('none')
            continue
        if chair_max > table_min: 
            print('below')
        elif chair_min < table_max:
            print('above')
        else:
            print('none')
    i += 1

Above
Image 0: below
Image 1: below
Image 2: none
Image 3: none
Image 4: none
Below
Image 0: none
Image 1: none
Image 2: below
Image 3: below
Image 4: below
Image 5: none
Impossible
Image 0: none
Image 1: below
Image 2: none
Image 3: none
Image 4: none
